In [ ]:
import tensorflow as tf
import matplotlib as plt
import matplotlib.pyplot as plt
import numpy as np
import keras
import cv2
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import PIL
import PIL.Image
from glob import glob
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from matplotlib import *
from matplotlib.pyplot import *
from keras.regularizers import l2
from numpy import *
import random
from tensorflow.keras.preprocessing.image import *
import scipy
from scipy import ndimage
from skimage.measure import label, regionprops

In [ ]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf) #inf = infinity 

# 함수 정리

### 일반 코드

- img_load_preproc(train_const_list) : 이미지 불러와서 array로 만들기
- for_label_index(test_label_array, i) : 인덱스 뽑아내기
- plot_data(train_data, num) : 데이터 하나씩 plot
- plot_wrong_classification(predicted_label, test_data, test_label_array) : 잘 분류된 것 16개
- plot_right_classification(predicted_label, test_data, test_label_array) : 잘못 분류된 것

### Grad CAM 관련 코드

- generate_gradCAM(model, last_conv_layer, last_dense_layer, train_data, num) : gradcam, img 리턴
- plot_gradCAM(model, last_conv_layer, last_dense_layer, train_data, num) : gradCAM 영상 보여줌


- wrong_classified_gradCAM(predicted_label, test_data, test_label_array) : 잘못 분류한 것들 대해서 원본 이미지와 gradCAM 보여줌


- generate_bbox(img, cam, threshold) : 빨간 range 생성하기용
- gradCAM_bbox(model, last_conv_layer, last_dense_layer, train_data, num) : gradCAM box 보여줌


- plot_gradCAM_set(model, last_conv_layer, last_dense_layer, train_data, num) : 원본&gradCAM&box 세개 보두 보여줌
- wrong_classified_gradCAM_set(predicted_label, test_data, test_label_array) : 잘못 분류된 영상 대해 모두 plot

# 함수들

In [ ]:
def img_load_preproc(train_const_list):
    train_const_data = []
    for file in train_const_list :
        img = cv2.imread(file, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (224,224))
        img = cv2.normalize(img, None, 0,255,cv2.NORM_MINMAX)
        train_const_data.append(img)
    return train_const_data

In [ ]:
#라벨값 뽑아내기
def for_label_index(test_label_array, i) :
    index = np.where(test_label_array[i]==np.max(test_label_array[i]))
    index = index[0]
    return index

In [ ]:
# 단순 데이터 하나씩 plot
def plot_data(train_data, num) :
    plt.imshow(cv2.cvtColor(train_data[num], cv2.COLOR_BGR2RGB))
    #train_label[num] = for_label_index(train_label, num)
    plt.title(for_label_index(train_label, num))
    plt.axis('off')
    plt.grid(b=None)

In [ ]:
#학습 진행할 때 정리해서 결과 띄우기
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(
            "Epoch {}\n "
            " The average loss (train/val) : {:7.7f} / {:7.7f}\n "
            " The average accuracy (train/val) : {:7.7f} / {:7.7f}\n "
            " The mean crossentropy (train/val) : {:7.7f} / {:7.7f}\n "
            "================================================================".format(
                epoch, 
                logs["loss"], logs["val_loss"],
                logs["accuracy"], logs["val_accuracy"],
                logs["binary_crossentropy"], logs["val_binary_crossentropy"]
            )
        )

In [ ]:
#학습 진행 후 결과 보기
def plot_acc_loss(history1_2):
    # check accuracy and loss
    plt.figure(figsize=[10,20])
    plt.subplot(3, 1, 1)
    plt.plot(history1_2.history['accuracy'], label='Training Accuracy')
    plt.plot(history1_2.history['val_accuracy'], label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.subplot(3, 1, 2)
    plt.plot(history1_2.history['loss'], label='Training Loss')
    plt.plot(history1_2.history['val_loss'], label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.subplot(3, 1, 3)
    plt.plot(history1_2.history['binary_crossentropy'], label='Training CrossEntropy')
    plt.plot(history1_2.history['val_binary_crossentropy'], label='Validation CrossEntropy')
    plt.legend(loc='upper right')
    plt.title('Training and Validation CrossEntropy')
    plt.show()

In [ ]:
# 분류 잘된 것들 뽑아내기(그냥 plot)
def plot_right_classification(predicted_label, test_data, test_label_array) :
    right_result = []
    for n in range(0, len(test_label_array)):
        if for_label_index(test_label_array, n) == for_label_index(predicted_label, n):
            right_result.append(n)
    sample_result = np.random.choice(len(right_result),16, replace=False)
    count=0
    nrows = ncols = 4
    plt.figure(figsize=(15,15))
    for n in sample_result:
        count += 1
        plt.subplot(nrows, ncols, count)
        plt.imshow(cv2.cvtColor(test_data[n], cv2.COLOR_BGR2RGB))
        tmp = "Label:" + str(for_label_index(test_label_array, n))+ \
        ", Prediction:" + str(for_label_index(predicted_label, n)) + ", " + str(np.max(predicted_label[n]))
        plt.title(tmp)
        plt.axis('off')
        plt.grid(b=None)
    plt.suptitle("Well Classified")
    plt.tight_layout()
    plt.show()
    print("well classified cases: ")
    print(len(right_result))

In [ ]:
# 분류 제대로 안된 것들 뽑아내기(그냥 plot)
def plot_wrong_classification(predicted_label, test_data, test_label_array) :
    wrong_result = []
    for n in range(0, len(test_label_array)):
        if for_label_index(test_label_array, n) != for_label_index(predicted_label, n):
            wrong_result.append(n)
    
    sample_result = np.random.choice(wrong_result,
                                     np.min([16, len(wrong_result)]),
                                     replace=False)
    count=0
    nrows = np.min([16, len(wrong_result)])
    ncols = 1
    plt.figure(figsize=(5,5*nrows))
    for n in sample_result:
        count += 1
        plt.subplot(nrows, ncols, count)
        plt.imshow(cv2.cvtColor(test_data[n], cv2.COLOR_BGR2RGB))
        tmp = "Label:" + str(for_label_index(test_label_array, n))+ \
        ", Prediction:" + str(for_label_index(predicted_label, n)) +", "+str(np.max(predicted_label[n]))
        plt.title(tmp)
        plt.axis('off')
        plt.grid(b=None)
    plt.suptitle("Wrong Classified")
    plt.tight_layout()
    plt.show()
    print("wrong classified cases: ")
    print(len(wrong_result))

In [ ]:
#gradCAM array로 뽑아내기
def generate_gradCAM(model, last_conv_layer, last_dense_layer, train_data, num) :
    
    """
    Grad CAM

    1. 마지막 conv layer output (feature map) 을 구한다 = last_conv_output (A_k)
    2. 마지막 dense layer output (logit) 을 구한다 (softmax 통과 전) = last_dense_output(y_c)
    3. logit을 feature map으로 편미분하여 weight를 구한다 = a_k (k는 마지막 conv층 filter)
    4. a_k에 각 필터별 feature map 곱하여 전체 필터 대한 결과값 더해준다.
    5. relu 통과시키면 끝
    """
    grad_model = keras.models.Model(inputs=model.inputs, 
                                outputs=[model.get_layer(last_conv_layer).output, 
                                         model.get_layer(last_dense_layer).output])
    with tf.GradientTape(persistent=False) as tape :
        A_k, y_c = grad_model((tf.expand_dims(train_data[num], axis=0)))
        loss = tf.squeeze(y_c[:, np.argmax(y_c[0])])

    grads = tape.gradient(loss, A_k)

    #가중치 계산. 피쳐맵의 각 피쳐가 로짓값에 얼마나 큰 영향 미치는지
    weights = np.mean(grads, axis = (0,1,2))
    
    # 추출한 conv_output에 weight를 곱하고 합하여 grad_cam을 얻는다.
    grad_cam = np.zeros(shape=A_k.shape[0:2])
    for k, w in enumerate(weights):
        grad_cam = grad_cam + w * A_k[0, :, :, k]    
    grad_cam = cv2.resize(np.array(grad_cam), (224, 224))

    ## ReLU를 씌워 음수를 0으로 만든다.
    grad_cam = np.maximum(grad_cam, 0)
    grad_cam = grad_cam / grad_cam.max()
    
    img = train_data[num]
    return grad_cam, img

In [ ]:
#gradCAM plot한 것까지 뽑아내기
def plot_gradCAM(model, last_conv_layer, last_dense_layer, train_data, num) :
    grad_model = keras.models.Model(inputs=model.inputs, 
                                outputs=[model.get_layer(last_conv_layer).output, 
                                         model.get_layer(last_dense_layer).output])
    with tf.GradientTape(persistent=False) as tape :
        A_k, y_c = grad_model((tf.expand_dims(train_data[num], axis=0)))
        loss = tf.squeeze(y_c[:, np.argmax(y_c[0])])

    grads = tape.gradient(loss, A_k)

    #가중치 계산. 피쳐맵의 각 피쳐가 로짓값에 얼마나 큰 영향 미치는지
    weights = np.mean(grads, axis = (0,1,2))
    
    # 추출한 conv_output에 weight를 곱하고 합하여 grad_cam을 얻는다.
    grad_cam = np.zeros(shape=A_k.shape[0:2])
    for k, w in enumerate(weights):
        grad_cam = grad_cam + w * A_k[0, :, :, k]    
    grad_cam = cv2.resize(np.array(grad_cam), (224, 224))

    ## ReLU를 씌워 음수를 0으로 만든다.
    grad_cam = np.maximum(grad_cam, 0)
    grad_cam = grad_cam / grad_cam.max()
    
    plt.figure(figsize=(10,10))
    plot_data(train_data, num)
    plt.imshow(grad_cam, cmap='jet', alpha=0.45)

In [ ]:
# 틀린 것만 골라내서 gradCAM 보기
def wrong_classified_gradCAM(predicted_label, test_data, test_label_array) :
    wrong_result = []
    for n in range(0, len(test_label_array)):
        if for_label_index(test_label_array, n) != for_label_index(predicted_label, n):
            wrong_result.append(n)
    
    sample_result = np.random.choice(wrong_result,
                                     np.min([16, len(wrong_result)]),
                                     replace=False)
    count=0
    nrows = np.min([16, len(wrong_result)])
    ncols = 2
    plt.figure(figsize=(10,20))
    for n in sample_result:
        count += 1
        plt.subplot(nrows, ncols, count)
        plt.imshow(cv2.cvtColor(test_data[n], cv2.COLOR_BGR2RGB))
        plt.imshow(generate_gradCAM(model1_1, 'conv2d_2', 'dense_1',test_data, n), cmap='jet', alpha=0.4)
        tmp = "Label:" + str(for_label_index(test_label_array, n))+ ", Prediction:" + str(for_label_index(predicted_label, n))
        plt.title(tmp)
        plt.axis('off')
        plt.grid(b=None)
        count += 1
        plt.subplot(nrows, ncols, count)
        plt.imshow(cv2.cvtColor(test_data[n], cv2.COLOR_BGR2RGB))
        tmp = "Label:" + str(for_label_index(test_label_array, n))+ ", Prediction:" + str(for_label_index(predicted_label, n))
        plt.title(tmp)
        plt.axis('off')
        plt.grid(b=None)
    plt.suptitle("Wrong Classified")
    plt.tight_layout()
    plt.show()

In [ ]:
# box 뽑아내기용 코드
def generate_bbox(img, cam, threshold):
    labeled, nr_objects = ndimage.label(cam > threshold)
    props = regionprops(labeled)
    return props

In [ ]:
#box plot 만
def gradCAM_bbox(model, last_conv_layer, last_dense_layer, train_data, num) :
    grad_cam, img = generate_gradCAM(model, last_conv_layer, last_dense_layer,train_data, num)
    props = generate_bbox(img, grad_cam , 0.1)
    
    fig,ax = plt.subplots(figsize=(5,5))
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    for b in props:
        bbox = b.bbox
        xs = bbox[1]
        ys = bbox[0]
        w = bbox[3] - bbox[1]
        h = bbox[2] - bbox[0]

        rect = patches.Rectangle((xs, ys), w, h, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    plt.axis('off')
    plt.grid(b=None)
    plt.show()

In [ ]:
#원본이미지, 그냥 gradCAM, box 까지 뽑아내기
def plot_gradCAM_set(model, last_conv_layer, last_dense_layer, train_data, num) :
    grad_cam, img = generate_gradCAM(model, last_conv_layer, last_dense_layer,train_data, num)
    props = generate_bbox(img, grad_cam , 0.2)
    
    fig,axes = plt.subplots(1,3,figsize=(15,5))
    
    axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[1].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[1].imshow(grad_cam, cmap='jet', alpha=0.4)
   
    axes[2].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    for b in props:
        bbox = b.bbox
        xs = bbox[1]
        ys = bbox[0]
        w = bbox[3] - bbox[1]
        h = bbox[2] - bbox[0]

        rect = patches.Rectangle((xs, ys), w, h, linewidth=2, edgecolor='r', facecolor='none')
        axes[2].add_patch(rect)

    axes[0].axis('off')
    axes[1].axis('off')
    axes[2].axis('off')    
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 틀린 것만 골라내서 gradCAM 보기
def wrong_classified_gradCAM_set(predicted_label, test_data, test_label_array) :
    wrong_result = []
    for n in range(0, len(test_label_array)):
        if for_label_index(test_label_array, n) != for_label_index(predicted_label, n):
            wrong_result.append(n)
    
    sample_result = np.random.choice(wrong_result,
                                     np.min([10, len(wrong_result)]),
                                     replace=False)

    for n in sample_result:
        plot_gradCAM_set(model1_1, 'conv2d_2', 'dense_1',test_data, n)
    plt.suptitle("Wrong Classified")
    plt.tight_layout()
    plt.show()

### 라벨 데이터 만들기

In [ ]:
test_num = 200
# label 만들기 : const 2850개, nonconst 3188개

const_label = [1., 0.]
nonconst_label = [0., 1.]

train_label=[]
test_label=[]

for i in range(0,2850-test_num):
    train_label.append(const_label) #2650개
for i in range(0,3188-test_num):
    train_label.append(nonconst_label) #2988개
    
for i in range(0, test_num):
    test_label.append(const_label)
for i in range(0,test_num):
    test_label.append(nonconst_label)
    
print(np.shape(train_label), np.shape(test_label))

### list 데이터 만들기 (train, test 따로)

In [ ]:
#const list 중 test data 고르기
data_const_list = glob('D:\\DSPL\\KT\\dataset\\Construction\\*.jpg') #2850개
train_const_list = []
test_const_list = []
np.random.seed(42)
for i in range(0,200) :
    choice = np.random.choice(data_const_list)
    test_const_list.append(choice)
    data_const_list.remove(choice)

train_const_list = data_const_list
print(len(train_const_list), len(test_const_list))

In [ ]:
#nonconst list 중 test data 고르기
data_nonconst_list = glob('D:\\DSPL\\KT\\dataset\\Road Vehicle\\*.png') #3188개
train_nonconst_list = []
test_nonconst_list = []
np.random.seed(54)
for i in range(0,200) :
    choice = np.random.choice(data_nonconst_list)
    test_nonconst_list.append(choice)
    data_nonconst_list.remove(choice)

train_nonconst_list = data_nonconst_list
print(len(train_nonconst_list), len(test_nonconst_list))

### data generate 하기

In [ ]:
# 0~255로 normalize
train_const_data = img_load_preproc(train_const_list)
train_nonconst_data = img_load_preproc(train_nonconst_list)
test_const_data = img_load_preproc(test_const_list)
test_nonconst_data = img_load_preproc(test_nonconst_list)

In [ ]:
print(np.shape(train_const_data), type(train_const_data))

In [ ]:
#cv2는 BGR인데 matplotlib은 RGB여서 변환해줘야 함.
plt.imshow(cv2.cvtColor(train_const_data[10], cv2.COLOR_BGR2RGB))

### train, test 별로 합치기

In [ ]:
train_data = np.concatenate((train_const_data, train_nonconst_data), axis=0)
test_data = np.concatenate((test_const_data, test_nonconst_data), axis=0)

### 데이터 셔플 (data, label)

In [ ]:
train_data = np.array(train_data)
train_label = np.array(train_label)

idx = np.arange(5638)
np.random.shuffle(idx)

for i in range (0,5638) :
    train_data[i] = train_data[idx[i]]
    train_label[i] = train_label[idx[i]]

plot_data(train_data, 130)

In [ ]:
plot_data(train_data,4)

# model train

In [ ]:
train_label_array = np.array(train_label)
test_label_array=np.array(test_label)

In [ ]:
l2=keras.regularizers.l2(1e-3)

model1_2 = tf.keras.Sequential()

model1_2.add(tf.keras.layers.Conv2D(filters = 16, strides = (1,1), kernel_size=(3,3), padding='same', 
                                    input_shape=(224,224,3), kernel_regularizer=l2))
model1_2.add(tf.keras.layers.BatchNormalization())
model1_2.add(tf.keras.layers.ReLU())
model1_2.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model1_2.add(tf.keras.layers.Conv2D(filters = 32, strides=(1,1), kernel_size=(3,3), padding="same",
                                   kernel_regularizer=l2))
model1_2.add(tf.keras.layers.BatchNormalization())
model1_2.add(tf.keras.layers.ReLU())
model1_2.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model1_2.add(tf.keras.layers.Conv2D(filters = 64, strides=(1,1), kernel_size=(3,3), padding="same",
                                   kernel_regularizer=l2))
model1_2.add(tf.keras.layers.BatchNormalization())
model1_2.add(tf.keras.layers.ReLU())
  
model1_2.add(tf.keras.layers.Flatten())
model1_2.add(tf.keras.layers.Dense(128, kernel_regularizer=l2))
model1_2.add(tf.keras.layers.BatchNormalization())
model1_2.add(tf.keras.layers.ReLU())
model1_2.add(tf.keras.layers.Dense(2))
model1_2.add(tf.keras.layers.Softmax())


lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01, 
                                                             decay_steps=100000,
                                                             decay_rate=1e-1, staircase=False)


model1_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                 loss=tf.keras.losses.BinaryCrossentropy(), 
                 metrics= ['accuracy', 'binary_crossentropy'])

In [ ]:
model1_2.summary()

In [ ]:
history1_2 = model1_2.fit(train_data, train_label_array, validation_split=0.2,
                          verbose=0, callbacks=[LossAndErrorPrintingCallback()],
                          batch_size=10, epochs=50)

In [ ]:
history1_2_1 = model1_2.fit(train_data, train_label_array, validation_split=0.2,
                          verbose=0, callbacks=[LossAndErrorPrintingCallback()],
                          batch_size=10, epochs=30)

In [ ]:
plot_acc_loss(history1_2)

In [ ]:
model1_2.save('KT_scene_classification_model1_2_epoch50.h5')

In [ ]:
model1_2.evaluate(test_data, test_label_array, batch_size=10)

In [ ]:
predicted_label_model1_2 = model1_2.predict(test_data)

In [ ]:
plot_wrong_classification(predicted_label_model1_2, test_data, test_label_array)

In [ ]:
wrong_classified_gradCAM_set(predicted_label_model1_2, test_data, test_label_array)

# 잘못 분류된 것 찾아보기 (model1_1)

In [ ]:
model1_1 = tf.keras.models.load_model('KT_scene_classification_model1_1_epoch30.h5')

In [ ]:
predicted_label_model1_1 = model1_1.predict(test_data, batch_size=10)

In [ ]:
plot_right_classification(predicted_label_model1_1, test_data, test_label_array)

In [ ]:
plot_wrong_classification(predicted_label_model1_1, test_data, test_label_array)

# Grad CAM

In [ ]:
plot_gradCAM(model1_1, 'conv2d_2', 'dense_1',train_data, 0)

In [ ]:
plot_gradCAM(model1_1, 'conv2d_2', 'dense_1', train_data, 1)

In [ ]:
plot_gradCAM(model1_1, 'conv2d_2', 'dense_1',train_data, 2)

In [ ]:
wrong_classified_gradCAM(predicted_label_model1_1, test_data, test_label_array)

In [ ]:
count=0
nrows=4
ncols=2
plt.figure(figsize=(10,20))
for n in range(0,8) :
    count = count+1
    plt.subplot(nrows, ncols, count)
    plt.imshow(cv2.cvtColor(test_data[n], cv2.COLOR_BGR2RGB))
    plt.imshow(generate_gradCAM(model1_1, 'conv2d_2', 'dense_1',test_data, n),cmap='jet', alpha=0.4)
    plt.axis('off')
    plt.grid(b=None)
    tmp = "Label:" + str(for_label_index(test_label_array, n))+ ", Prediction:" + str(for_label_index(predicted_label_model1_1, n))
    plt.title(tmp)
plt.tight_layout()
plt.show()

In [ ]:
plot_gradCAM_set(model1_1, 'conv2d_2', 'dense_1',test_data, 207)

In [ ]:
wrong_classified_gradCAM_set(predicted_label_model1_1, test_data, test_label_array)

In [ ]:
for n in range(0,20) :
    plot_gradCAM_set(model1_1, 'conv2d_2', 'dense_1',test_data, n)

In [ ]:
for n in range(200,220) :
    plot_gradCAM_set(model1_1, 'conv2d_2', 'dense_1',test_data, n)

In [ ]:
for n in range(300,320) :
    plot_gradCAM_set(model1_1, 'conv2d_2', 'dense_1',test_data, n)